<a href="https://colab.research.google.com/github/rzagni/Fintech/blob/main/news_rag_sp500_zagniA01795457.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**<font color="green">Fintech and Digital Innovation in Finance**
###**<font color="green">Course project part 3 | Retrieval Augmented Generation with S&P 500 news data**
###**Renzo Zagni A01795457**</font>


# Introduction to Retrieval Augmented Generation with S&P 500 news

In this notebook, you will explore how to build a simple Retrieval-Augmented Generation (RAG) pipeline using financial news articles from S&P 500 companies.

We'll start by vectorizing text data, creating a vector store using FAISS, and integrating it with OpenAI's GPT models to answer questions using retrieved information.

This workflow emulates real-world systems in finance where natural language data (news, filings, analyst reports) are used to support decision-making.

# 📌 Objectives

By the end of this notebook, students will be able to:

1. **Perform Semantic Search with Metadata Filtering:**
   - Query the provided FAISS vector store to retrieve relevant financial news articles based on natural language questions.
   - Apply optional filters using metadata such as ticker or publication date to refine search results.

2. **Enrich Data with Company Metadata:**
   - Use the `yfinance` library to retrieve company-level metadata (company name, sector, industry) for tickers in the dataset.
   - Integrate this metadata to support enhanced filtering and analysis of news data.

3. **Build a Retrieval-Augmented Generation (RAG) Pipeline:**
   - Combine retrieved news snippets as context to generate answers using OpenAI’s GPT models.
   - Construct effective prompts that guide the language model to provide concise, context-aware responses.

4. **Evaluate and Analyze RAG Outputs:**
   - Review generated answers alongside the supporting news excerpts.
   - Reflect on the strengths and limitations of the simple RAG pipeline and consider potential improvements, such as adding more filters or refining retrieval strategies.

5. **Incorporate Financial Metadata into Retrieval Context:**
   - Enrich retrieved news snippets with key financial metadata including ticker, company name, sector, and industry.
   - Format prompts that combine both text excerpts and metadata to provide richer context to the language model.

6. **Generate Context-Aware Answers Using OpenAI Models:**
   - Construct and send prompts to an LLM that leverage both news content and metadata to produce concise, informed financial analysis.

7. **Compare Answers With and Without Metadata:**
   - Evaluate the impact of including financial metadata on answer quality using criteria such as clarity, detail, accuracy, and contextual relevance.
   - Summarize findings to reflect on the role of metadata in improving retrieval-augmented generation.

## Install and Import important librairies

First, we install and import the necessary libraries for:
- Text embedding generation (sentence-transformers)
- Efficient similarity search (faiss)
- Data manipulation (pandas, numpy)
- Visualization (matplotlib)

> ℹ️ FAISS uses inner product for cosine similarity by normalizing vectors.

In [ ]:
%pip install sentence-transformers
%pip install faiss-cpu

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.21.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvjitlink_cu

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt
import faiss

In [ ]:
# Additional libraries used
import os
import time
import pickle
from IPython.display import clear_output
from datetime import datetime
import yfinance as yf
import time

## Load news data
We load a CSV file of financial news, focusing on TITLE and SUMMARY, along with metadata like TICKER and PUBLICATION_DATE.
These will be embedded into vectors and used for semantic retrieval.

In [ ]:
#
# Connect to G Drive to save and retrieve objects
from google.colab import drive
drive.mount('/content/drive')
working_dir = '/content/drive/MyDrive/MNA/Fintech'

os.chdir(working_dir)
print(f"Working directory: {working_dir}")

Mounted at /content/drive
Working directory: /content/drive/MyDrive/MNA/Fintech


In [ ]:
K = 25

In [ ]:
#
# df_news.csv was loaded in google drive under {working_dir}
df_news = pd.read_csv('df_news.csv')
df_news['PUBLICATION_DATE'] = pd.to_datetime(df_news['PUBLICATION_DATE']).dt.date
display(df_news)

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...
...,...,...,...,...,...,...
4866,ZTS,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis is a leading animal health company with...,2025-05-23,Motley Fool,https://www.fool.com/investing/2025/05/23/2-di...
4867,ZTS,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis (NYSE:ZTS) recently affirmed a dividend...,2025-05-22,Simply Wall St.,https://finance.yahoo.com/news/zoetis-nyse-zts...
4868,ZTS,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,We recently published a list of Jim Cramer Tal...,2025-05-21,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-zoet...
4869,ZTS,Zoetis (ZTS) Upgraded to Buy: Here's Why,Zoetis (ZTS) might move higher on growing opti...,2025-05-21,Zacks,https://finance.yahoo.com/news/zoetis-zts-upgr...


In [ ]:
df_news['EMBEDDED_TEXT'] = df_news['TITLE'] + ' : ' + df_news['SUMMARY']

In [ ]:
#
# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Implement FAISS vector store
We:
- Use a pre-trained sentence transformer (all-MiniLM-L6-v2) to embed documents.
- Normalize vectors to use cosine similarity.
- Create a FAISS index and implement a basic search function.

This will allow us to retrieve relevant news snippets given a natural language question.


In [ ]:
# Load model and compute embeddings
text_embeddings = model.encode(df_news['EMBEDDED_TEXT'].tolist(), convert_to_numpy=True)

# Normalize embeddings to use cosine similarity (via inner product in FAISS)
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)

# Prepare metadata
documents = df_news['EMBEDDED_TEXT'].tolist()
metadata = [
    {
        'PUBLICATION_DATE': row['PUBLICATION_DATE'],
        'TICKER': row['TICKER'],
        'PROVIDER': row['PROVIDER']
    }
    for _, row in df_news.iterrows()
]

In [ ]:
embedding_dim = text_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)  # Cosine similarity via inner product
faiss_index.add(text_embeddings)

In [ ]:
class FaissVectorStore:
    def __init__(self, model, index, embeddings, documents, metadata):
        self.model = model
        self.index = index
        self.embeddings = embeddings
        self.documents = documents
        self.metadata = metadata

    def search(self, query, k=5, metadata_filter=None):
        # Encode and normalize query as float32
        query_embedding = self.model.encode([query], normalize_embeddings=True).astype("float32")

        results = []
        if metadata_filter:
            # Filter metadata and cap k
            filtered_indices = [i for i, meta in enumerate(self.metadata) if metadata_filter(meta)]
            if not filtered_indices:
                return []
            filtered_embeddings = self.embeddings[filtered_indices]
            temp_index = faiss.IndexFlatIP(filtered_embeddings.shape[1])
            temp_index.add(filtered_embeddings)
            k_eff = int(min(k, len(filtered_indices)))
            D, I = temp_index.search(query_embedding, k_eff)
            D0, I0 = D[0], I[0]
            # Map local indices back to global indices
            indices = [filtered_indices[i] for i in I0]
            for idx, sim in zip(indices, D0):
                if idx < 0 or idx >= len(self.documents):
                    continue
                results.append((self.documents[idx], self.metadata[idx], float(sim)))
            return results
        else:
            # Cap k to index size
            k_eff = int(min(k, self.index.ntotal))
            D, I = self.index.search(query_embedding, k_eff)
            D0, I0 = D[0], I[0]
            for idx, sim in zip(I0, D0):
                if idx < 0 or idx >= len(self.documents):
                    continue
                results.append((self.documents[idx], self.metadata[idx], float(sim)))
            return results


In [ ]:
# Create FAISS-based store
faiss_store = FaissVectorStore(
    model=model,
    index=faiss_index,
    embeddings=text_embeddings,
    documents=documents,
    metadata=metadata
)

### Setup OpenAI Client

👉 **Instructions**:
- Import the `OpenAI` client from the `openai` Python library.
- You will need an **OpenAI API key** to use their models programmatically:
  - Go to [https://platform.openai.com/](https://platform.openai.com/) and sign up or log in.
  - Create an API key from your [API keys dashboard](https://platform.openai.com/account/api-keys).
  - ⚠️ **Keep your API key private** and **do not** share or hardcode it in public notebooks.
- Note that **usage of the OpenAI API is not free**. You will need to:
  - Add a payment method.
  - Monitor your usage to avoid unexpected charges.
  - Optionally set usage limits from your account settings.
- You can refer to the **course’s Study Resources** for a step-by-step guide on creating an OpenAI account and retrieving your API key.

Then:
- Initialize the client with `OpenAI(api_key="YOUR_KEY_HERE")`.
- Send a test request using `.responses.create()` and the `"gpt-4o-mini"` model with a simple prompt:

  ```python
  response = client.responses.create(
      model="gpt-4o-mini",
      input="Write a one-sentence bedtime story about a unicorn."
  )
  print(response.output_text)


In [ ]:
#
# Install openai libraries
!pip install openai

In [ ]:
#
# import openai library
from openai import OpenAI

In [ ]:
#
# Initialize openai client
api_key  = ''

client = OpenAI(
  api_key=api_key
)

In [ ]:
#
# Test openai api key and connectivity
response = client.responses.create(
    model="gpt-4o-mini",
    input="Write a one-sentence bedtime story about a unicorn."
)
print(response.output_text)

In a shimmering meadow under a starlit sky, a playful unicorn sprinkled dreams of kindness and joy, ensuring that every child slept soundly with a smile.


## Retrieve Additional Metadata from Yahoo Finance

👉 **Instructions**:
- We will enrich our news dataset by retrieving **company-level metadata** using the `yfinance` library.
- The goal is to map each unique stock ticker (`TICKER`) in the dataset to:
  - `COMPANY_NAME`
  - `SECTOR`
  - `INDUSTRY`

> ℹ️ `yfinance` fetches live data from Yahoo Finance. If you're running this in a cloud environment or during peak hours, expect some tickers to fail or rate limits to apply.

✅ After this step, you will have a new DataFrame (e.g. `df_meta`) with the columns `TICKER`, `COMPANY_NAME`, `SECTOR`, `INDUSTRY` that maps tickers to their company names, sectors, and industries. This metadata will be useful later to add filters and analysis based on sector or industry categories.


In [ ]:
#
# Let's get the list of S&P 500 tickers from yfinance
# Read and print the stock tickers that make up S&P500
df_tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

display(df_tickers.head())

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [ ]:
#
# Let's put the tickers on a list and address the known issue with dots in the ticker name
ticker_list = df_tickers['Symbol'].tolist()
ticker_list = [ticker.replace('.', '-') for ticker in ticker_list]
print(f"Number of Tickers Downloaded: {len(ticker_list)}")

Number of Tickers Downloaded: 503


In [ ]:
#
# Note: on Aug/7th Skydance (PSKY) adquired Paramount (PARA) causing it's ticker
# to be remove from S&P 500. To address the inconsistencies between the newly
# generated list of S&P500 company (post Aug-7) and the news retrieve prior
# we will remove news entries related to Paramount from the news dataframe
# Alternatively, we could repeat the ticker news download but since this is a long
# running process and since it will not affect the objectives and results of this
# exercise we chose to reuse the news in the existing dataframe
new_SP500 = set(df_tickers['Symbol'])
df_news = df_news[df_news['TICKER'].isin(new_SP500)]

In [ ]:
#
# Let's build the df_meta dataframe with the following columns TICKET, COMPANY_NAME, SECTOR, INDUSTRY

def get_metadata(ticker_list, delay=0.5):
    """
    Get company metadata for a list of stock tickers using yfinance (no retry).

    Parameters
    ----------
    ticker_list : list of stock ticker
    delay : float, default=0.5
        Delay in seconds between API calls to prevent timeouts or errors

    Returns
    -------
    db_meta : pd.DataFrame
        DataFrame with columns: TICKET, COMPANY_NAME, SECTOR, INDUSTRY.
    failed : list of str
        Tickers that failed to fetch.
    """
    rows = []
    failed = []

    for idx, ticker in enumerate(ticker_list, 1):
        clear_output(wait=True)
        print("Downloading metadata from Yahoo Finance...")
        print(f"Processing: {ticker} ({idx} of {len(ticker_list)})")

        try:
            stock = yf.Ticker(ticker)
            info = stock.get_info()

            rows.append({
                "TICKER": ticker,
                "COMPANY_NAME": info.get('longName'),
                "SECTOR": info.get('sector'),
                "INDUSTRY": info.get('industry')
            })

        except Exception:
            failed.append(ticker)

        time.sleep(delay)

    db_meta = pd.DataFrame(rows, columns=["TICKER", "COMPANY_NAME", "SECTOR", "INDUSTRY"])
    return db_meta, failed


In [ ]:
#
# In case we already ran this notebook and want to reuse the dictionary created
# during previous executions
df_meta_pickle = 'meta.pkl'
df_meta = None  # will store our dataframe

if not os.path.isfile(df_meta_pickle):
    # Get metadata from yfinance
    db_meta, failed = get_metadata(ticker_list)

    if failed:
        print(f"Tickers failed to fetch metadata: {failed}")
    else:
        print("All tickers processed successfully!")

    # Save to pickle
    with open(df_meta_pickle, 'wb') as f:
        pickle.dump(db_meta, f)

else:
    print("Metadata already downloaded. Loading from local file...")
    with open(df_meta_pickle, 'rb') as f:
        df_meta = pickle.load(f)


Metadata already downloaded. Loading from local file...


In [ ]:
df_meta.head()

,TICKER,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,3M Company,Industrials,Conglomerates
1,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General
4,ACN,Accenture plc,Technology,Information Technology Services


## Retrieval-Augmented Generation (RAG): Retrieve Documents and Generate Answers

👉 **Instructions**:

In this part of the assignment, your task is to build a simple Retrieval-Augmented Generation (RAG) pipeline that:

- Takes a user question as input.
- Searches the FAISS vector store to find a set of relevant financial news articles based on semantic similarity.
- Uses the retrieved news articles as context to generate a clear, concise answer to the question by interacting with the OpenAI language model.
- Returns both the generated answer and the underlying news snippets used for context.

### What you need to focus on:

- Implement a retrieval mechanism to query your vector store and obtain the top relevant documents for any question.
- Construct prompts that effectively combine retrieved news content with the user’s question to guide the language model’s response.
- Use the OpenAI API to generate answers grounded in the retrieved context.
- Organize the outputs so that for each question, you have:
  - The generated answer.
  - The collection of news excerpts used to produce that answer.

### What you will be provided:

- Helper functions to display outputs in markdown format.
- Lists of example questions covering topics, companies, and industries to test your implementation.

---

Your solution can take any form or structure you find appropriate, as long as it fulfills these core objectives. This exercise will give you hands-on experience with integrating retrieval and generation for practical applications in finance.


#### Print markdown
You can use the following function to print answers from GPT4o-mini in markdown.

In [ ]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

#### Predefined questions

In [ ]:
questions_topic = [
"What are the major concerns expressed in financial news about inflation?",
"How is investor sentiment described in recent financial headlines?",
"What role is artificial intelligence playing in recent finance-related news stories?"
]

questions_company = [
"How is Microsoft being portrayed in news stories about artificial intelligence?",
"What financial news headlines connect Amazon with automation or logistics?"
]

questions_industry = [
"What are the main themes emerging in financial news about the semiconductor industry?",
"What trends are being reported in the retail industry?",
"What risks or challenges are discussed in recent news about the energy industry?"
]

In [ ]:
def _val(meta, *keys, default=""):
    """
    Return the first non-None value from meta for any of the given keys, else default.
    """
    for k in keys:
        if isinstance(meta, dict) and k in meta and meta[k] is not None:
            return meta[k]
    return default


def _days_since(dstr):
    """
    Return days elapsed since ISO date string dstr, or 9999 on error
    """
    try:
        return (datetime.utcnow().date() - datetime.fromisoformat(str(dstr)).date()).days
    except Exception:
        return 9999


def retrieve(question, store, k=3, overfetch=None):
    """
    Search store for question, dedupe, rerank by similarity and recency, and
    return top k results with metadata.
    """
    K = overfetch or max(12, k*3)
    hits = store.search(question, k=K)

    seen, rows = set(), []
    for doc, meta, sim in hits:
        title = _val(meta, "title", "TITLE")
        date  = _val(meta, "date", "PUBLICATION_DATE")
        key = (title.strip(), str(date).strip()) if title or date else (
            _val(meta, "row_idx", "ROW_IDX", default=None),
            _val(meta, "url", "URL", default=None)
        )
        if key in seen:
            continue
        seen.add(key)
        rows.append((doc, meta, float(sim)))

    scored = []
    for doc, meta, sim in rows:
        date = _val(meta, "date", "PUBLICATION_DATE")
        days = _days_since(date)
        rec  = max(0.0, 1.0 - min(days, 365)/365.0)
        final = 0.85*sim + 0.15*rec
        scored.append((final, doc, meta, sim))
    scored.sort(reverse=True, key=lambda x: x[0])

    contexts = []
    for final, doc, meta, sim in scored[:k]:
        contexts.append({
            "title":    _val(meta, "title", "TITLE"),
            "date":     _val(meta, "date", "PUBLICATION_DATE"),
            "source":   _val(meta, "source", "PROVIDER", default="news"),
            "ticker":   _val(meta, "ticker", "TICKER", "TICKER_NORM"),
            "company":  _val(meta, "company", "COMPANY_NAME"),
            "sector":   _val(meta, "sector", "SECTOR"),
            "industry": _val(meta, "industry", "INDUSTRY"),
            "url":      _val(meta, "url", "URL", default=None),
            "full":     doc,
            "score":    float(final),
        })
    return contexts


In [ ]:
def custom_prompt(question, contexts):
    """
    Format contexts and question into system and user messages for a financial
    news analysis prompt.
    """
    # Build context lines
    ctx_lines = []
    for i, c in enumerate(contexts, start=1):
        ticker   = c.get("ticker", "")
        company  = c.get("company", "")
        sector   = c.get("sector", "")
        industry = c.get("industry", "")
        score    = c.get("score", 0.0)

        # Use full text as snippet
        snippet = (c.get("full") or "").strip()

        label = " - ".join(x for x in [ticker, company] if x)
        meta  = ", ".join(x for x in [sector, industry] if x)

        ctx_lines.append(f"[{i}] {label} ({meta}) — score {score:.3f}\n{snippet}")

    context_block = "\n\n".join(ctx_lines) if ctx_lines else "(no context retrieved)"

    system_msg = (
        "You are a financial news analyst. Respond strictly using only the provided "
        "context. Summaries must be concise, in 4–7 well-structured sentences, focusing "
        "on facts, key figures, and trends. If the context lacks the necessary information, "
        "state this explicitly without speculating. Avoid repetition, personal opinions, "
        "or unsupported claims. Maintain a neutral, professional financial tone at all times."
    )

    user_msg = (
        f"Context:\n{context_block}\n\n"
        f"Question: {question}\n"
        "Instructions: Provide a grounded answer."
    )
    return system_msg, user_msg



In [ ]:
def show_result(question, contexts, answer):
    """
    Display the question, answer, and supporting contexts in formatted markdown.
    """
    print_markdown(f"###Question:\n{question}")
    print_markdown(f"###Answer:\n{answer}")
    print_markdown(f"###Context used:\n")
    for c in contexts:
        label = f"{c['ticker']} - {c['company']}"
        meta_line = f"{c['sector']}, {c['industry']}"
        snippet = c['full']
        print_markdown(f"####{label} ({meta_line}) — score {c['score']:.3f}\n\n")
        print_markdown(f"   {snippet}\n")



In [ ]:
def generate_answer(
    question,
    contexts,
    *,
    openai_model="gpt-4o-mini",
    temperature=0.2,
    max_tokens=350,
):
    """
    Generate a grounded answer to a question using provided contexts via an OpenAI model.
    """
    sys_msg, usr_msg = custom_prompt(question, contexts)

    resp = client.responses.create(
        model=openai_model,
        temperature=temperature,
        max_output_tokens=max_tokens,
        input=[
            {"role": "system", "content": [{"type": "input_text", "text": sys_msg}]},
            {"role": "user",   "content": [{"type": "input_text", "text": usr_msg}]},
        ],
    )
    return resp.output_text.strip()

In [ ]:
def rag(question, faiss_store, k=8):
    """
    Executes RAG pipeline: retrieve context, generate answer, and display results.
    """
    ctx = retrieve(question, faiss_store, k=k)
    ans = generate_answer(question, ctx)
    show_result(question, ctx, ans)

In [ ]:
#
# Questions Topic
print_markdown("#Topic")
for q in questions_topic:
    rag(q, faiss_store, k=K)

#Topic

###Question:
What are the major concerns expressed in financial news about inflation?

###Answer:
Financial news highlights significant concerns regarding persistent U.S. inflation and its potential impact on economic growth. The Federal Reserve's recent minutes indicate rising worries about inflation risks, suggesting that these concerns could lead to a slowdown in economic activity. Additionally, food inflation is dampening expectations for interest rate cuts, as ongoing tariff issues contribute to economic uncertainty. Overall, the macroeconomic outlook remains cautious, with analysts adjusting earnings expectations downward due to these inflationary pressures.

###Context used:


####BLK -  (, ) — score 0.610



...

   With EPS Growth And More, Jack Henry & Associates (NASDAQ:JKHY) Makes An Interesting Case : Investors are often guided by the idea of discovering 'the next big thing', even if that means buying 'story stocks...


####TKO -  (, ) — score 0.456



   High Growth Tech Stocks To Watch In The US April 2025 : As the U.S. stock market navigates through a period of mixed performance, with major indices like the Dow Jones experiencing fluctuations due to ongoing tariff uncertainties, investors are closely watching the tech sector for potential opportunities. In this environment, identifying high-growth tech stocks involves looking for companies that demonstrate resilience and innovation amidst economic challenges and market volatility.


####DAY -  (, ) — score 0.454



   3 Stocks Estimated To Be Trading Below Their Intrinsic Value In April 2025 : Over the last 7 days, the United States market has risen 5.2%, contributing to a 9.9% increase over the past year, with earnings forecasted to grow by 14% annually. In this environment of growth, identifying stocks that are trading below their intrinsic value can offer potential opportunities for investors seeking to capitalize on market inefficiencies.


In [ ]:
#
# Questions Company
print_markdown("#Company")
for q in questions_company:
    rag(q,faiss_store, k=K)

#Company

###Question:
How is Microsoft being portrayed in news stories about artificial intelligence?

###Answer:
Microsoft is portrayed positively in the context of artificial intelligence, particularly following its recent earnings report, which has contributed to its stock attempting to break past a new buy point. The company is recognized as a key player in the AI market, alongside other major firms like Google and NVIDIA, which are leading innovations in AI technologies. The overall sentiment suggests that Microsoft is leveraging its AI capabilities to enhance its competitive position in the software sector. However, specific details regarding its AI investments or strategic initiatives are not provided in the context.

###Context used:


####META -  (, ) — score 0.581



...

   Top Stock Reports for Alphabet, Berkshire Hathaway & Visa : Today's Research Daily features new research reports on 16 major stocks, including Alphabet Inc. (GOOGL), Berkshire Hathaway Inc. (BRK.B) and Visa Inc. (V), as well as a micro-cap stock bioAffinity Technologies, Inc. (BIAF).


####JKHY -  (, ) — score 0.469



   With EPS Growth And More, Jack Henry & Associates (NASDAQ:JKHY) Makes An Interesting Case : Investors are often guided by the idea of discovering 'the next big thing', even if that means buying 'story stocks...


####LIN -  (, ) — score 0.456



   Stocks to watch this week: Amazon, Palantir, Disney, Novo Nordisk and AstraZeneca : Earnings preview of key companies reporting this week and what to look out for.


In [ ]:
#
# Questions Industry
print_markdown("#Industry")
for q in questions_industry:
    rag(q, faiss_store,  k=K)

#Industry

###Question:
What are the main themes emerging in financial news about the semiconductor industry?

###Answer:
The financial news surrounding the semiconductor industry highlights several key themes. ON Semiconductor Corporation (ON) has garnered significant attention due to its international revenue trends and recent stock performance, including a 30% price surge despite reporting a net loss for Q1 2025. Investor interest is also reflected in ON's ongoing share buyback program, which may have bolstered confidence amid challenging earnings. Additionally, there are concerns regarding declining demand in electric vehicles (EVs), which could impact growth, particularly in sectors like silicon carbide (SiC) and AI data centers. Overall, the semiconductor sector appears to be navigating a complex landscape of investor optimism and macroeconomic challenges.

###Context used:


####ON -  (, ) — score 0.659



...

   Grim Economic Outlook Overtakes Solid Earnings as Tariff Disruptions Surface : (Bloomberg) -- One thing is clear as the first-quarter earnings season draws to a close: The uncertain outlook for the global economy is superseding better-than-feared results even as stocks rally on signs of easing trade tensions.Most Read from BloombergAs Coastline Erodes, One California City Considers ‘Retreat Now’How a Highway Became San Francisco’s Newest ParkMaryland’s Credit Rating Gets Downgraded as Governor Blames Trump America, ‘Nation of Porches’Power-Hungry Data Centers Are Warming H


####DVN -  (, ) — score 0.450



   This Top Oil Stock's Smart Plan Puts It in a Stronger Position to Weather Volatile Crude Oil Prices : Devon Energy delivered better-than-expected oil production in the first quarter.  Its smart business optimization plan will boost its free cash flow by $1 billion by the end of 2026.  Devon Energy (NYSE: DVN) has spent several years building a larger-scale, low-cost U.S. onshore oil and gas producer.


####GEHC -  (, ) — score 0.447



   Earnings updates: Ford, Stellantis, GE Healthcare, & more : Tariff talk is heating up again this earnings season as companies sound the alarm on trade-related uncertainty. Stellantis (STLA) suspended its 2025 guidance, citing tariff uncertainty, while GE Healthcare (GEHC) cut its outlook due to global trade pressures. Stanley Black & Decker (SWK) says it's weighing pricing changes and Ford (F) earnings on Monday will also be closely watched. To watch more expert insights and analysis on the latest market action, check out more Market Domination&nbsp;here.


## Analysis & Questions - Section 1

### Analysis and Reflection on Retrieval and Generation Results
After running the RAG pipeline and obtaining answers along with their supporting news excerpts, take some time to carefully review both the generated responses and the retrieved contexts.

- **For each question, read the answer and then the corresponding news snippets used as context.**

- Reflect on the following points and document your observations:
1. **Relevance**
2. **Completeness**  
3. **Bias or Noise**
4. **Consistency**  
5. **Improvement Ideas**   

and answer the questions below:

#### **Question 1.** How well do the retrieved news snippets support the generated answer? Are the key facts or themes in the answer clearly grounded in the context?

<font color="green">The results are mixed. Most answer are readable and on-topic, but several lean on broad statements or toss in precise stats without showing where they came from. That makes them sound confident, yet less trustworthy.</font>

<font color="green">The best: the AI-focused answers for “AI in finance” and “Amazon automation or logistics”. They give concrete company names, actions, and plausible connections such as AI rollouts, logistics moves. They’re specific enough to be useful and don’t overreach much. If every answer hit this balance—clear claims tied to clear examples—you’d be in good shape.</font>

<font color="green">Not good nor bad: “Investor sentiment,” “semiconductors,” and “energy risks.” These read logically and capture real themes such optimism vs. headwinds; ON/SiC/AI data centers,  renewables policy and oilfield services, but they feel general market commentary that could apply anytime such as "Recent financial headlines indicate a mixed sentiment among investors". </font>

<font color="green">Not so good: “Inflation,” “Microsoft & AI,” and “retail trends.” The inflation and retail pieces include exact numbers without evidence; the Microsoft answer makes sweeping claims about partnerships without clear, MSFT-specific substance. </font>


#### **Question 2.** Does the answer fully address the question, or does it leave important aspects out? Consider if the retrieved context provided enough information to generate a thorough response.

<font color="green">Some of answers feel truly complete the others are not. When the news were specific and on-topic, the generated answers were solid. On the other hand, when the context was broad or made up of roundup/listicle pieces, the answers drifted into generic market talk or filled gaps with assumptions.</font>

<font color="green">In the TOPIC set, the “AI in finance” answer is solid—it names specific companies and actions that actually show up in the material, so it addresses the question well. The “investor sentiment” answer has the right tone : optimism vs. caution.</font>

<font color="green">In the COMPANY set, Amazon + automation/logistics was the question that has the best answer .The reference to AWS’s Matt Garman on AI across Amazon gave enough facts to answer the question cleanly. The Microsoft & AI answer, though, talks up partnerships and leadership without MSFT-centric material in the context. It reads more like a general impression than a response grounded in Microsoft-specific coverage, so it doesn’t fully address the question.</font>

<font color="green">Across the INDUSTRY set, completeness varies. The semiconductor answer leans heavily on ON Semiconductor and mentions SiC/AI data centers, but misses other live threads such as Trump tariffs,  export controls, Nvidia/TSMC and China dynamics). This may be a symptom of narrow retrieval, so the answer feels narrow too. The same applies to Retail were it  only touches performance pressure and tariff/inventory angles. Energy covers renewables policy risk and pressure on oilfield services, but leaves out other usual movers OPEC, Iran, Tensions in the Middle East, etc. </font>

<font color="green">In general, the context was enough to produce thorough answers for Amazon/logistics and a good one for AI-in-finance; it was too thin or too generic for Microsoft, parts of inflation, semis, retail, and energy. To get more complete answers, tighten retrieval (filter by ticker/sector, prioritize recent primary stories over roundups), and have generation anchor claims to 1–2 specifics from the retrieved items. If the context doesn’t support a point, the answer should say so rather than filling --hallucinating-- in with extra stats.</font>
  

#### **Question 3.** Are there any irrelevant or misleading snippets retrieved that may have influenced the answer? How might this affect the quality of the output?

<font color="green">Yes, there’s quite a bit of off-target and roundup content in the retrieved sets, and it does shape the answers. The clearest case is the Microsoft question. The context is dominated by stories about Meta, Salesforce, Jack Henry, and general AI market pieces. The only Microsoft mentions are incidental, for instance a ServiceNow headline that lists Microsoft among several partners and the Marvell piece tied to Amazon’s Trainium. With so little MSFT-specific substance, the answer drifts into broad claims about “partnerships” and “leadership” without concrete Microsoft evidence.</font>

<font color="green">We also see noise in the Amazon item: while two snippets are spot-on , the news about AWS CEO Matt Garman on AI across Amazon, and Stord buying UPS’s Ware2Go, the rest includes unrelated analyst roundups and sector news : Workday, Emerson, and OKE. </font>

<font color="green">The industry questions show the quality hit more clearly. The semiconductor set is heavy on ON Semiconductor plus several non-semi items (ETR, BK, JKHY, OKE, PKG), which narrows the answer to ON/SiC/AI data centers and crowds out other core chip themes . The energy set mixes relevant renewables policy and oilfield-services pieces with “Industrials down 10.4%” headlines; the answer repeats that industrials stat in an energy context—technically sourced, but misleading for the question asked.</font>

<font color="green">Even the inflation topic blends a Fed-minutes article with listicles and a gold-bullish piece that drops a national-debt/interest line. That kind of snippet can skew the answer toward dramatic macro stats rather than the core drivers in mainstream inflation coverage: wages, shelter, expectations. These resulted in irrelevant or loosely related snippets that made answers seem more generic, more selective with numbers, and sometimes off-topic. </font>


#### **Question 4.**  Do the news snippets show consistent information, or are there conflicting viewpoints? How does the LLM handle potential contradictions in the context?

<font color="green">The snippets mix aligned themes with conflicting signals. We can see both sides in several sets. For investor sentiment, many headlines are upbeat (price targets, momentum), while others flag “headwinds” and rare bearish calls. Semiconductors show a direct conflict on ON Semi: one piece touts a 30% one-month surge, another notes it plunged 35% YTD; both can be true, but they’re different windows. Energy mixes risk-heavy stories, such as renewables-subsidy threat and  pressure on oilfield services, with “sector up today” news, which point in opposite near-term directions. Retail performance numbers also vary because the articles use different sub-industries and timeframes.</font>

<font color="green">Contradictions were averaged away or selectively emphasized. That keeps the prose tidy, which blurred important nuances like timeframes or scopes and that  lead to inconsisten answers.</font>

<font color="green">To make the LLM  handle conflicts better we could  instruct it to identify and reconcile differences explicitly , for example “If sources disagree, name both and explain timeframe/coverage differences” andrequire snippet attributions for any numbers or strong claims. </font>
  

#### **Question 5.**  Based on your observations, suggest ways the retrieval or generation process could be improved (e.g., better filtering, adjusting `k`, refining prompt design).

<font color="green"> To improvide the quality of the answers, we could clean up the data sent to the  model by starting with the removal noisy sources like listicles and generic analyst blogs. Use one time window so we don’t mix short-term moves with year-to-date trends. For company questions, we should only keep articles that name the ticker in the headline or the first sentence. For topic and industry questions, we should include stories about several different companies so one name doesn’t dominate. And make sure every snippet has full metadata—ticker, company name, sector, industry, title, date, and source—so labels are clear and you don’t get blanks in the prompt.</font>

<font color="green">On retrieval, the system should overfetch and then prune. Pulling a larger candidate set, deduplicating similar items, and re-ranking by direct relevance to the question and exact ticker matches can also improve the quality of the answer.</font>

<font color="green">The size of the final context (“k”) should be tuned to the task. Topic and industry prompts benefit from a slightly larger, more diverse set to capture cross-currents; company prompts read better with fewer, tighter, on-ticker articles. In practice: retrieve broadly, then keep only a small, non-redundant, high-relevance subset.</font>

<font color="green">Lastly , and as I mentioned in my previous answer , refining the prompt could help generating higher quality answers by making it grounded, specific, and consistent across TOPIC, COMPANY, and INDUSTRY. The system message should set strict rules: use only provided snippets, cite them, handle conflicts, and the user message should give a clean, labeled context block the model can point to. We should also pair this with low temperature (~0.2–0.3) and a short max length so the model doesn’t wander.</font>

<font color="green"> This is an example of a tigher prompt:</font>

<pre>SYSTEM
You are a financial news analyst. Use only the snippets provided below.
Rules:
1) No outside facts. If a fact isn’t in the snippets, write: “The provided articles don’t include enough detail to answer that.”
2) After any number, ticker-specific claim, or direct quote, add the snippet index in brackets , for example [2].
3) If snippets conflict, state both views and reconcile by timeframe/scope (e.g., “up MoM, still down YTD”).
4) Keep it concise . Use 4 to 6 sentences,be  neutral, and specific. Avoid generic language.

Category guidance:
- TOPIC: Cover at least 3 distinct tickers if present; call out the dominant driver(s).
- COMPANY: Focus on the named ticker; include at least 2 company-specific facts with citations.
- INDUSTRY: Cover 2–4 drivers and 1–2 risks; avoid over-indexing on a single firm.

Forbidden: invented numbers, un-cited claims, “according to some reports,” or speculation.
</pre>

## 🧠 Retrieval-Augmented Generation (RAG) v2: Adding Financial Metadata to Improve Generation

👉 **Instructions**:

In this part of the assignment, you’ll enhance your Retrieval-Augmented Generation (RAG) pipeline by incorporating *financial metadata* to provide more contextually rich answers.

Your goal is to evaluate whether metadata such as **company name**, **sector**, and **industry** helps the LLM generate **more accurate and grounded answers** to financial questions.

---

### ✅ What your updated pipeline should do:

- Retrieve relevant financial news articles using semantic similarity with FAISS.
- Enrich each retrieved document with financial metadata:
  - Ticker symbol
  - Full company name
  - Sector (e.g., Technology, Energy)
  - Industry (e.g., Semiconductors, Retail)
- Construct prompts that include both:
  - Retrieved news text
  - Associated metadata
- Send the prompt to the OpenAI model to generate an informed response.
- Return:
  - The final answer
  - The exact set of contextual documents used to produce that answer

---

### 🧪 Evaluation and Comparison:

You will test your improved RAG pipeline on the same three types of questions provided earlier:
- **Topic-focused** (e.g., inflation, interest rates)
- **Company-focused** (e.g., questions about Tesla, Nvidia)
- **Industry-focused** (e.g., semiconductors, utilities)


In [ ]:
def norm(sym: str) -> str:
    return str(sym).strip().upper().replace('.', '-')

df_news = df_news.copy()
df_news["TICKER_NORM"] = df_news["TICKER"].map(norm)

df_meta = df_meta.copy()
df_meta["TICKER_NORM"] = df_meta["TICKER"].map(norm)

#
# Let's merge company metadata into each news row
cols_meta = ["TICKER_NORM", "COMPANY_NAME", "SECTOR", "INDUSTRY"]
df_newsm = df_news.merge(df_meta[cols_meta], on="TICKER_NORM", how="left", suffixes=("", "_m"))

# (Optional) show coverage:
covered = df_newsm["COMPANY_NAME"].notna().mean()
print(f"Metadata coverage: {covered:.1%} of news rows have company info.")

Metadata coverage: 100.0% of news rows have company info.


In [ ]:
documents = df_newsm["EMBEDDED_TEXT"].astype(str).tolist()

# Metadata dict for each doc (same order, same length)
metadata = [
    {
        "title": row["TITLE"],
        "date": str(row["PUBLICATION_DATE"]),
        "source": row.get("PROVIDER", "news"),
        "summary": row["SUMMARY"],
        "ticker": row["TICKER_NORM"],
        "company": row.get("COMPANY_NAME"),
        "sector": row.get("SECTOR"),
        "industry": row.get("INDUSTRY"),
        "url": row.get("URL"),
        "row_idx": int(i),
    }
    for i, row in df_newsm.iterrows()
]
len(documents), len(metadata)

(4841, 4841)

In [ ]:
text_embeddings_aug = model.encode(documents, normalize_embeddings=True).astype("float32")
faiss_index_aug = faiss.IndexFlatIP(text_embeddings_aug.shape[1])
faiss_index_aug.add(text_embeddings)

print("docs:", len(documents), "meta:", len(metadata),
      "emb:", text_embeddings_aug.shape[0], "idx:", faiss_index.ntotal)



# Instantiate FaissVectorStore with augmented index and embeddings
faiss_store_augmented = FaissVectorStore(
    model=model,
    index=faiss_index_aug,
    embeddings = text_embeddings_aug,
    documents=documents,
    metadata=metadata
)

docs: 4841 meta: 4841 emb: 4841 idx: 4871


In [ ]:
#
# Questions Topic
print_markdown("#Topic")
for q in questions_topic:
    rag(q, faiss_store_augmented, k=K)

#Topic

###Question:
What are the major concerns expressed in financial news about inflation?

###Answer:
The financial news highlights several major concerns regarding inflation, particularly in relation to potential tariffs and their impact on supply chains. Bristol-Myers Squibb's CEO emphasized that tariffs targeting the pharmaceutical industry could lead to supply constraints, which may exacerbate inflationary pressures. Additionally, the broader economic outlook remains uncertain, prompting analysts to lower earnings estimates across various sectors, indicating a cautious approach to inflation's potential effects. The recent downgrade of the U.S. sovereign credit rating by Moody's has also raised concerns about fiscal health, further complicating the inflation narrative. Overall, the interplay between tariffs, supply chain disruptions, and economic forecasts is central to the ongoing inflation discourse.

###Context used:


####BX - Blackstone Inc. (Financial Services, Asset Management) — score 0.607



...

   Workday (WDAY)’s AI Push and Margin Beat Lead to $250 Price Target : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Workday, Inc. (NASDAQ:WDAY) stands against other AI stocks on Wall Street’s radar. Workday, Inc. (NASDAQ:WDAY) provides enterprise cloud applications. On May 23, DA Davidson raised the firm’s price target on […]


####OTIS - Otis Worldwide Corporation (Industrials, Specialty Industrial Machinery) — score 0.516



   Otis Worldwide (OTIS) Up 4.8% Since Last Earnings Report: Can It Continue? : Otis Worldwide (OTIS) reported earnings 30 days ago. What's next for the stock? We take a look at earnings estimates for some clues.


####PH - Parker-Hannifin Corporation (Industrials, Specialty Industrial Machinery) — score 0.512



   Earnings Beat: Parker-Hannifin Corporation Just Beat Analyst Forecasts, And Analysts Have Been Updating Their Models : Investors in Parker-Hannifin Corporation ( NYSE:PH ) had a good week, as its shares rose 3.5% to close at US$619...


In [ ]:
#
# Questions Company
print_markdown("#Company")
for q in questions_company:
    rag(q,faiss_store_augmented, k=K)

#Company

###Question:
How is Microsoft being portrayed in news stories about artificial intelligence?

###Answer:
The provided context does not contain specific information regarding Microsoft's portrayal in news stories about artificial intelligence. Therefore, I cannot provide an answer based on the available data.

###Context used:


####MTD - Mettler-Toledo International Inc. (Healthcare, Diagnostics & Research) — score 0.575



...

   Those who invested in Public Service Enterprise Group (NYSE:PEG) five years ago are up 80% : If you buy and hold a stock for many years, you'd hope to be making a profit. Better yet, you'd like to see the share...


####EIX - Edison International (Utilities, Utilities - Regulated Electric) — score 0.507



   Tragic Wildfires in Southern California Affected Edison International (EIX) in Q1 : ClearBridge Investments, an investment management company, released its “ClearBridge Large Cap Value Strategy” first quarter 2025 investor letter. A copy of the letter can be downloaded here. The S&P 500 Index declined by -4.3% in Q1 2025 due to a tariff war and a shift away from AI-related tech stocks. Amid the tech-led sell-off, the […]


####CDNS - Cadence Design Systems, Inc. (Technology, Software - Application) — score 0.507



   US Tightens China Chip Curbs by Targeting Design Software : (Bloomberg) -- The Trump administration is moving to restrict the sale of chip design software to China, people familiar with the matter said, as the US government evaluates a broader policy announcement on the issue.Most Read from BloombergNYC Congestion Toll Brings In $216 Million in First Four MonthsNY Wins Order Against US Funding Freeze in Congestion FightNY Congestion Pricing Is Likely to Stay Until Year End During Court CaseNow With Colorful Blocks, Tirana’s Pyramid Represents a Changing


In [ ]:
#
# Questions Industry
print_markdown("#Industry")
for q in questions_industry:
    rag(q, faiss_store_augmented,  k=K)

#Industry

###Question:
What are the main themes emerging in financial news about the semiconductor industry?

###Answer:
The financial news regarding the semiconductor industry highlights several key themes. Firstly, there is a focus on stock performance, with companies like Analog Devices and ON Semiconductor being scrutinized for their earnings and market positioning. For instance, ON Semiconductor has experienced a significant 35% decline year-to-date, attributed to challenging macroeconomic conditions despite strong growth in SiC and AI data centers.

Additionally, there is an emphasis on strategic moves within the industry, such as acquisitions and partnerships, which are seen as critical for growth and competitiveness. The overall market for semiconductors is also being influenced by broader economic factors, including trade policies and tariffs, which could impact inventory levels and supply chains. Lastly, the integration of artificial intelligence into semiconductor applications is a recurring topic, indicating a shift towards next-gen computing solutions.

###Context used:


####ORCL - Oracle Corporation (Technology, Software - Infrastructure) — score 0.665



...

   Pool Corporation (NASDAQ:POOL) Looks Like A Good Stock, And It's Going Ex-Dividend Soon : Readers hoping to buy Pool Corporation ( NASDAQ:POOL ) for its dividend will need to make their move shortly, as the...


####CRWD - CrowdStrike Holdings, Inc. (Technology, Software - Infrastructure) — score 0.487



   CrowdStrike Holdings, Inc. (CRWD): A Bear Case Theory : We came across a bearish thesis on CrowdStrike Holdings, Inc. (CRWD) on Stock Whisperer’s Substack. In this article, we will summarize the bears’ thesis on CRWD. CrowdStrike Holdings, Inc. (CRWD)’s share was trading at $455.59 as of 23rd May. CRWD’s trailing and forward P/E were 401.83 and 131.58 respectively according to Yahoo Finance. CrowdStrike Holdings (CRWD), a […]


####PCAR - PACCAR Inc (Industrials, Farm & Heavy Construction Machinery) — score 0.487



   Upwork and PACCAR have been highlighted as Zacks Bull and Bear of the Day : Upwork and PACCAR have been highlighted as Zacks Bull and Bear of the Day.


## Analysis & Questions - Section 2

### Instructions: Evaluate Answers With and Without Metadata

For each question, compare the two answers provided:
- One generated **without** metadata
- One generated **with** metadata

---

### Steps:

1. Use the following evaluation criteria:
   - Clarity
   - Detail & Depth
   - Use of Context
   - Accuracy & Grounding
   - Relevance
   - Narrrative Flow

2. For each criterion, write brief notes comparing how the answer **without metadata** performs versus the answer **with metadata**.

3. Summarize your evaluation in a markdown table with the following columns:

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [Your brief note here]     | [Your brief note here]   |
| Detail & Depth         | [Your brief note here]     | [Your brief note here]   |
| Use of Context        | [Your brief note here]     | [Your brief note here]   |
| Accuracy & Grounding       | [Your brief note here]     | [Your brief note here]   |
| Relevance      | [Your brief note here]     | [Your brief note here]   |
| Narrative Flow      | [Your brief note here]     | [Your brief note here]   |

---

**Note:** Keep comments short and clear for easy comparison.



| <font color="green"><b>Criteria</b></font> | <font color="green">WITHOUT METADATA</font> | <font color="green">WITH METADATA</font> |
|----------------------------------------------|------------------------------------------------------|---------------------------------------------------|
| <font color="green"><b>Clarity</b></font> | <font color="green">Answers are understandable but can feel vague because they lack concrete identifiers like company names or dates.</font> | <font color="green">Provides immediate clarity by referencing specific companies, sectors, and industries, allowing readers to understand exactly who and what is being discussed.</font> |
| <font color="green"><b>Detail & Depth</b></font> | <font color="green">Focuses on broad market themes or trends without drilling into the specifics, resulting in surface-level observations that are less insightful.</font> | <font color="green">Adds layers of information such as sector performance, industry trends, and company-level details, making the responses more nuanced and informative.</font> |
| <font color="green"><b>Use of Context</b></font> | <font color="green">Relies on generic summaries that are loosely connected to the question, which can weaken the contextual grounding of the answer.</font> | <font color="green">Context is tightly linked to real-world entities, events, and timelines, which strengthens the connection between the provided evidence and the answer.</font> |
| <font color="green"><b>Accuracy & Grounding</b></font> | <font color="green">Because examples are generic, it’s easier for the model to make slight inaccuracies or assumptions without being noticed by the reader.</font> | <font color="green">Accuracy improves because claims are explicitly tied to named companies, industries, and factual details, reducing the risk of vague or unverified statements.</font> |
| <font color="green"><b>Relevance</b></font> | <font color="green">Sometimes drifts into tangential topics, especially when the retrieved context is broad and lacks strong entity matches.</font> | <font color="green">Maintains high relevance by filtering context to match entities directly tied to the query, keeping the focus tightly on the question.</font> |
| <font color="green"><b>Narrative Flow</b></font> | <font color="green">Reads like a stitched collection of general facts, with minimal connective tissue between points, making it less engaging.</font> | <font color="green">Narrative is smoother and more cohesive because company and sector references give the answer a clear storyline with logical progression.</font> |
